# Getting started with periodic systems
[Link to tutorial](https://octopus-code.org/documentation/13/tutorial/periodic_systems/periodic_systems/)

The extension of a ground-state calculation to a periodic system is quite straightforward in **Octopus**. In this tutorial we will explain how to perform some basic calculation using bulk silicon as an example.

In [ ]:
import subprocess

import matplotlib.pyplot as plt
import pandas as pd

from postopus import Run

In [ ]:
pd.set_option("display.max_rows", 10)

In [ ]:
!mkdir -p ./1-getting-started

In [ ]:
cd 1-getting-started

### Input

As always, we will start with a simple input file. In this case we will use a primitive cell of Si, composed of two atoms.

In [ ]:
%%writefile nk.oct


nk = 2

In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs

PeriodicDimensions = 3

Spacing = 0.5

a = 10.18
%LatticeParameters
 a | a | a
%

%LatticeVectors
 0.0 | 0.5 | 0.5
 0.5 | 0.0 | 0.5
 0.5 | 0.5 | 0.0
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

include nk.oct
%KPointsGrid
  nk |  nk |  nk
 0.5 | 0.5 | 0.5
 0.5 | 0.0 | 0.0
 0.0 | 0.5 | 0.0
 0.0 | 0.0 | 0.5
%
KPointsUseSymmetries = yes

ExtraStates = 1
%Output
 dos
%

Lets see more in detail some of the input variables:


* [PeriodicDimensions](https://www.octopus-code.org/documentation//13/variables/system/periodicdimensions) = 3: this input variable must be set equal to the number of dimensions you want to consider as periodic. Since the system is 3D ([Dimensions](https://www.octopus-code.org/documentation//13/variables/system/dimensions)) = 3 is the default), by setting this variable to 3 we impose periodic boundary conditions at all borders. This means that we have a fully periodic infinite crystal.


* [LatticeVectors](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/latticevectors) and [LatticeParameters](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/latticeparameters): these two blocks are used to define the primitive lattice vectors that determine the unit cell. [LatticeVectors](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/latticevectors) defines the direction of the vectors, while [LatticeParameters](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/latticeparameters) defines their length.


* [ReducedCoordinates](https://www.octopus-code.org/documentation//13/variables/system/coordinates/reducedcoordinates): the position of the atoms inside the unit cell, in reduced coordinates.


* [KPointsGrid](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointsgrid): this specifies the ''k''-point grid to be used in the calculation. Here we employ a 2x2x2 Monkhorst-Pack grid with four shifts. The first line of the block defines the number of ''k''-points along each axis in the Brillouin zone. Since we want the same number of points along each direction, we have defined the auxiliary variable `nk = 2` This will be useful later on to study the convergence with respect to the number of ''k''-points. The other four lines define the shifts, one per line, expressed in reduced coordinates of the Brillouin zone. Alternatively, one can also define the reciprocal-space mesh by explicitly setting the position and weight of each ''k''-point using the [KPoints](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpoints) or [KPointsReduced](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointsreduced) variables.


* [KPointsUseSymmetries](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointsusesymmetries) = yes: this variable controls if symmetries are used or not. When symmetries are used, the code shrinks the Brillouin zone to its irreducible portion and the effective number of ''k''-points is adjusted.


* [Output](https://www.octopus-code.org/documentation//13/variables/output/output) = dos: we ask the code to output the density of states.


Here we have taken the value of the grid spacing to be 0.5 bohr. Although we will use this value throughout this tutorial, remember that in a real-life calculation the convergence with respect to the grid spacing must be performed for all quantities of interest.
Note that for periodic systems the default value for the [Boxshape](https://www.octopus-code.org/documentation//13/variables/mesh/simulation_box/boxshape) variable is parallelepiped, although in this case the name can be misleading, as the actual shape also depends on the lattice vectors. This is the only box shape currently available for periodic systems.



### Output

Now run **octopus** using the above input file. 

In [ ]:
!octopus

Here are some important things to note from the output.

In [ ]:
!cat stdout_gs.txt | grep -A 4 "[*] Space [*]"

This tells us that out system is indeed being treated as periodic in 3 dimensions.

In [ ]:
!cat stdout_gs.txt | grep -A 10 "[*] Grid [*]"

Here **octopus** outputs some information about the cell in real and reciprocal space.

In [ ]:
!cat stdout_gs.txt | grep -A 31 "[*] Symmetries [*]"

This block tells us about the space-group and the symmetries found for the specified structure.

In [ ]:
!cat stdout_gs.txt | grep -A 15 "[*] Lattice [*]"

Here **Octopus** outputs some information about the unit cell in real and reciprocal space.

In [ ]:
!cat stdout_gs.txt | grep -A 21 "Checking if the generated full k-point"

Next we get the list of the ''k''-points in reduced coordinates and their weights. Since symmetries are used, only two ''k''-points are generated. If we had not used symmetries, we would have 32 ''k''-points instead.

The rest of the output is much like its non-periodic counterpart. After a few iterations the code should converge:

In [ ]:
!cat stdout_gs.txt | grep -A 21 "SCF CYCLE ITER #   12"

As usual, the static/info file contains the most relevant information concerning the calculation. Since we asked the code to output the density of states, we also have a few new files in the static directory:

* dos-XXXX.dat : the band-resolved density of states (DOS);
* total-dos.dat : the total DOS (summed over all bands);
* total-dos-efermi.dat : the Fermi Energy in a format compatible with total-dos.dat .

Of course you can tune the output type and format in the same way you do in a finite-system calculation.

## Convergence in k-points

Similar to the convergence in spacing, a convergence must be performed for the sampling of the Brillouin zone. To do this one must try different numbers of ‘‘k’'-points along each axis in the Brillouin zone. This can easily be done by changing the value of the nk auxiliary variable in the previous input file. You can obviously do this by hand, but this is something that can also be done with a script. Here is such a script.

In [ ]:
def get_energy(directory):
    """
    Extract Total energy from info.
    """
    info = Run(directory).default.scf.info()

    total_energy = [line for line in info if "Total       = " in line][-1]
    total_energy = float(total_energy.split("=")[-1])
    return total_energy

In [ ]:
list_of_k_points = [
    2,
    4,
    6,
    8,
    10,
    12,
]
table = []

for k_points in list_of_k_points:
    with open("nk.oct", "w") as f:
        # set number of k points
        f.write(f"nk = {k_points}")
    # run octopus
    print(f"Running octopus for number of k-points: {k_points}")

    subprocess.run("octopus", shell=True, cwd=".")

    # extract output
    total_energy = get_energy(".")
    table.append((k_points, total_energy))

total_energy_df = pd.DataFrame(table, columns=["k points", "Total Energy"])
print(total_energy_df)

As you can see, the total energy is converged to within 0.0001 hartree for nk = 6.

You can now play with an extended range, e.g. from 2 to 12. You should then see that the total energy is converged to less than a micro hartree.

## Band-structure

We now proceed with the calculation of the band-structure of Si. In order to compute a band-structure, we must perform a non-self-consistent calculation, using the density of a previous ground-state calculation. So the first step is to obtain the initial ground-state. To do this, rerun the previous input file, but changing the number of k-points to nk = 6. Next, modify the input file such that it looks like this:

Recalculate the ground state with nk = 6

In [ ]:
%%writefile nk.oct


nk = 6

Calculate the band structure

In [ ]:
%%writefile inp

stdout = 'stdout_unocc.txt'
stderr = 'stderr_unocc.txt'

CalculationMode = unocc

PeriodicDimensions = 3

Spacing = 0.5

a = 10.18
%LatticeParameters
 a | a | a
%

%LatticeVectors
 0.0 | 0.5 | 0.5
 0.5 | 0.0 | 0.5
 0.5 | 0.5 | 0.0
%

%ReducedCoordinates
 "Si" | 0.0 | 0.0 | 0.0
 "Si" | 1/4 | 1/4 | 1/4
%

ExtraStates = 10
ExtraStatesToConverge = 5

%KPointsPath
  10 |  10 |  15
 0.5 | 0.0 | 0.0  # L point
 0.0 | 0.0 | 0.0  # Gamma point
 0.0 | 0.5 | 0.5  # X point
 1.0 | 1.0 | 1.0  # Another Gamma point
%
KPointsUseSymmetries = no

Here are the things we changed:

* [CalculationMode]() = unocc: we are now performing a non-self-consistent calculation, so we use the unoccupied calculation mode;

* [ExtraStates]() = 10: this is the number of unoccupied bands to calculate;

* [ExtraStatesToConverge]() = 5: the highest unoccupied states are very hard to converge, so we use this variable to specify how many unoccupied states are considered for the stopping criterion of the non-self-consistent run.

* [KPointsPath](): this block is used to specify that we want to calculate the band structure along a certain path in the Brillouin zone. This replaces the KPointsGrid block. The first row describes how many ‘‘k’'-points will be used to sample each segment. The next rows are the coordinates of the ‘‘k’'-points from which each segment starts and stops. In this particular example, we chose the following path: L-Gamma, Gamma-X, X-Gamma using a sampling of 10-10-15 ‘‘k’'-points.

* [KPointsUseSymmetries]() = no: we have turned off the use of symmetries.



In [ ]:
!octopus

After running **Octopus** with this input file, you should obtain a file named bandstructure inside the static directory. This is how the first few lines of the file should look like:

In [ ]:
run = Run(".")
bandstructure = run.default.scf.bandstructure()
bandstructure

The first column is the coordinate of the ‘‘k’'-point along the path. The second, third, and fourth columns are the reduced coordinates of the ‘‘k’'-point. The following columns are the eigenvalues for the different bands. In this case there are 14 bands (4 occupied and 10 unoccupied).

In [ ]:
fig, axs = plt.subplots()
purple_bands = [f"band_{i}" for i in range(1, 4 + 1)]
green_bands = [f"band_{i}" for i in range(5, 10 + 1)]
bandstructure.plot(y=purple_bands, ax=axs, color="purple", linewidth=0.7, legend=False)
bandstructure.plot(y=green_bands, ax=axs, color="green", linewidth=0.7, legend=False)
axs.set_ylabel("E (hartree)")
fig.suptitle(
    "Band structure of bulk silicon. The zero of energy has been shifted to the maximum of the occupied bands."
);

Above you can see the plot of the band structure. This plot shows the occupied bands (purple) and the first 5 unoccupied bands (green). Note that when using the [KPointsPath](https://www.octopus-code.org/documentation//13/variables/mesh/kpoints/kpointspath) input variable, **Octopus** will run in a special mode, and the restart information of the previous ground-state calculation will not be altered in any way. The code informs us about this just before starting the unoccupied states iterations:

Info: The code will run in band structure mode.
     No restart information will be printed.

[Go to *2-Wires-and-slabes.ipynb*](2-Wires-and-slabes.ipynb)